<html>
<table width="100%" cellspacing="2" cellpadding="2" border="1">
<tbody>
<tr>
<td valign="center" align="center" width="45%"><img src="../media/Univ-Utah.jpeg"><br>
</td>
    <td valign="center" align="center" width="75%">
<h1 align="center"><font size="+1">University of Utah<br>Population Health Sciences<br>Data Science Workshop</font></h1></td>
<td valign="center" align="center" width="45%"><img
src="../media/U_Health_stacked_png_red.png" alt="Utah Health
Logo" width="128" height="134"><br>
</td>
</tr>
</tbody>
</table>
<br>
</html>

In [2]:
# from helpers import *
import medspacy
import pandas as pd

from medspacy.visualization import visualize_dep, visualize_ent, MedspaCyVisualizerWidget

In [3]:
import warnings
warnings.filterwarnings("ignore") 

# NLP Exercises
The last few notebooks stepped through the major parts of a clinical NLP system. In this notebook you'll build a complete system for identifying [pneumonia](https://en.wikipedia.org/wiki/Pneumonia) in radiology reports.

## 0. Load the data
The dataset in these examples is a set of MIMIC-II radiology reports. The annotations were created by University of Utah physician-scientist and pneumonia extraordinaire [Dr. Barbara Jones](https://healthcare.utah.edu/fad/mddetail.php?physicianID=u0102859&name=barbara-e-jones). As baseline to compare our system against we will use a system recently developed by her team for identifying misdiagnosis of pneumonia in clinical notes: [`medspacy_pna`](https://github.com/abchapman93/medspacy_pneumonia). This was system was designed for VA and University of Utah data, so it might not achieve as high of performance on MIMIC data as what is reported in the paper. Let's see if we can beat its performance!

The data is split into two sets: the **training set** and **testing set**. We'll start by developing our system with the training set before doing a final evaluation on the testing set.

### 0.3 Read in the data
Run the code below to read in the training set. The resulting dataframe will have a column for:
- The document name
- The text
- The annotator's document classification (this is the **"truth"**)
- The baseline NLP system's document classification (this is the **"prediction"**)

We'll eventually add another column with our own predictions.

In [3]:
df = read_pneumonia_data("train")

Reading data from: ../data/pneumonia_data_train.json


In [4]:
df.head()

,filename,text,document_classification,baseline_document_classification
0,subject_id_6383_hadm_id_19146,\n\n\n DATE: [**2717-5-23**] 12:06 PM\n ...,0,0
1,subject_id_4516_hadm_id_16710,\n\n\n DATE: [**2900-2-18**] 3:17 PM\n ...,0,0
2,subject_id_283_hadm_id_17902,\n\n\n DATE: [**2948-10-10**] 7:32 PM\n ...,1,1
3,subject_id_7068_hadm_id_28746,\n\n\n DATE: [**3043-3-19**] 9:22 AM\n ...,0,0
4,subject_id_4726_hadm_id_27535,\n\n\n DATE: [**2904-8-20**] 4:47 PM\n ...,0,0


## 1. Document annotation
Before building an NLP system we need to define our concepts annotate a corpus of notes to use as a reference standard. We already have an annotated corpus, so we'll review a few short examples and see how we would annotate them and then look at the reference standard annotations that we already have.

### 1.1
For this task, we will define a **"POS"** note as: 

*A note which contains a positive **or** possible mention of a term referring to pneumonia.*

Consider the following terms to be pneumonia:

- Pneumonia
- Pna
- Opacity
- Infiltrate
- Consolidation

Review the following notes and annotate each as 1 for positive or 0 for negative.

In [5]:
# RUN CELL TO SEE QUIZ
quiz_pna_annotation1

In [6]:
# RUN CELL TO SEE QUIZ
quiz_pna_annotation2

In [7]:
# RUN CELL TO SEE QUIZ
quiz_pna_annotation3

In [8]:
# RUN CELL TO SEE QUIZ
quiz_pna_annotation4

### 1.2
The true annotations can be found in `df["document_classification"]`. Answer the questions below.

In [9]:
# RUN CELL TO SEE QUIZ
quiz_num_pna_notes

In [10]:
len(df)

140

In [11]:
# RUN CELL TO SEE QUIZ
quiz_num_pos_pna_notes

In [13]:
df["document_classification"].mean()

0.4857142857142857

## 2. Build your NLP system and process texts
Now that we have some idea about what our dataset contains, let's starting building an NLP system and reviewing the output. First, build an empty NLP system. Then we'll process the notes in our dataset using our system as is (which doesn't have any rules). Go through the output and review the data. Find some examples of pneumonia that you should extract. Then go through and add rules for each of the following components as needed:

1. Add target concept rules to `target_matcher` to identify pneumonia in the text
2. Add ConText rules to `context` to improve attribute assertion
3. Optionally, add additional rules to `sectionizer` if the section logic is helpful for classifying the entities.
4. Build a document classifier which returns `0` or `1` for a doc. A simple version would just use the ConText attributes like `is_negated`, but a more complex version might also use information such as the section of the note.
5. Evaluate the system and review errors

After adding rules, reprocess your notes and review the output again. Since NLP is a computationally expensive procedure, you might want to work in batches of 10 or so before processing the whole corpus.

### 2.1: Load a model
Import `medspacy` and create an `nlp` model.

In [14]:
nlp = medspacy.load()

In [15]:
nlp.pipe_names

['medspacy_pyrush', 'medspacy_target_matcher', 'medspacy_context']

In [16]:
# RUN CELL TO TEST VALUE
test_load_nlp.test(nlp)

That is correct!


### 2.2: Add a Sectionizer
By default, medspaCy doesn't load a `Sectionizer` component, but we want to include section detection in our pipeline. Add a sectionizer to your pipeline.

In [17]:
nlp.add_pipe("medspacy_sectionizer")

In [18]:
test_load_nlp_add_sectionizer.test(nlp)

That is correct!


### 2.3: Process notes
The code below will process the notes in `df_train` with your NLP. If you want to use the whole dataset, set `df_train = df`. Otherwise, work in batches like `df_train = df.iloc[:10]`, `df_train = df.iloc[10:20]`, etc.

In [19]:
# df_train = df
df_train = df.iloc[:10]

In [43]:
docs = list(nlp.pipe(df_train["text"]))

In [44]:
w = MedspaCyVisualizerWidget(docs)

Box(children=(HBox(children=(RadioButtons(options=('Ent', 'Dep', 'Both'), value='Ent'), Button(description='Pr…

### 2.4 Concept extraction
Add rules to the `target_matcher` component to extract mentions of pneumonia.

In [45]:
from medspacy.target_matcher import TargetRule
target_rules = [
    TargetRule("PNA", "PNEUMONIA"),
    TargetRule("pneumonia", "PNEUMONIA"),
     TargetRule("PNA","PNEUMONIA"),
    TargetRule("pneumonia","PNEUMONIA"),
    TargetRule("opacities", "PNEUMONIA"),
    TargetRule("opacification", "PNEUMONIA"),
    TargetRule("opacity", "PNEUMONIA"),
    TargetRule("infiltrate", "PNEUMONIA"),
    TargetRule("inflitrates", "PNEUMONIA"),
    TargetRule("consolidation", "PNEUMONIA"),
]
if len(target_rules) > 0:
    nlp.get_pipe("medspacy_target_matcher").add(target_rules)

### 2.5 ConText
Add any modifiers which were not captured with the default rule set.

In [ ]:
from medspacy.context import ConTextRule
context_rules = [

]
if len(context_rules) > 0:
    nlp.get_pipe("medspacy_context").add(context_rules)

### 2.6 Sections
Add any section titles which were not detected and led to errors.

In [ ]:
from medspacy.section_detection import Sectionizer
section_rules = [

]

if len(section_rules) > 0:
    nlp.get_pipe("medspacy_sectionizer").add(section_rules)

### 2.7: Document Classification
Write a function called `classify_pna` which takes a doc and returns a `1` if the document is positive for pneumonia and `0` if it is negative.

In [29]:
def classify_pna(doc):
    for ent in doc.ents:
        if not (ent._.is_negated or ent._.is_uncertain or ent._.is_family or ent._.is_historical or ent._.is_hypothetical):
            return 1
    return 0

In [30]:
pred_1 = classify_pna(nlp("There is no evidence of pna"))

In [31]:
pred_1

0

In [32]:
# RUN CELL TO TEST VALUE
test_classify_pna_1.test(pred_1)

That is correct!


In [37]:
pred_2 = classify_pna(nlp("Impression: pneumonia"))

In [38]:
pred_2

1

In [39]:
# RUN CELL TO TEST VALUE
test_classify_pna_2.test(pred_2)

That is correct!


### 2.8: Evaluate your system on training data
After reprocessing your texts and creating `docs` with an updated NLP, run the code below to get performance metrics for your system. The function `evaluate_system` will return a DataFrame with performance characteristics for your system as well as the baseline system.

Look at the results and ask the following questions:
- What sorts of mistakes does my system appear to be making?
- Is precision or recall higher? What does that mean in the context of the research question?
- How is it comparing to the baseline NLP?

In [40]:
# Add your predictions
df_train = add_document_classifications(df_train, docs, classify_pna)

In [41]:
df_train.head()

,filename,text,document_classification,baseline_document_classification,nlp_document_classification,doc
0,subject_id_6383_hadm_id_19146,\n\n\n DATE: [**2717-5-23**] 12:06 PM\n ...,0,0,0,"(\n\n\n , DATE, :, [, *, *, 2717, -, 5, -,..."
1,subject_id_4516_hadm_id_16710,\n\n\n DATE: [**2900-2-18**] 3:17 PM\n ...,0,0,0,"(\n\n\n , DATE, :, [, *, *, 2900, -, 2, -,..."
2,subject_id_283_hadm_id_17902,\n\n\n DATE: [**2948-10-10**] 7:32 PM\n ...,1,1,1,"(\n\n\n , DATE, :, [, *, *, 2948, -, 10, -..."
3,subject_id_7068_hadm_id_28746,\n\n\n DATE: [**3043-3-19**] 9:22 AM\n ...,0,0,0,"(\n\n\n , DATE, :, [, *, *, 3043, -, 3, -,..."
4,subject_id_4726_hadm_id_27535,\n\n\n DATE: [**2904-8-20**] 4:47 PM\n ...,0,0,0,"(\n\n\n , DATE, :, [, *, *, 2904, -, 8, -,..."


In [42]:
results_train = evaluate_system(df_train)
results_train

,system,accuracy,precision,recall,f1-score,n_positive,n_total
0,baseline,0.8,1.0,0.6,0.750000,5,10
1,nlp,0.6,1.0,0.2,0.333333,5,10


### 2.9: Error Analysis
Review examples of mistakes your NLP system made. We'll subset the dataframe to look at **false positives** and **false negatives**.

First, edit the code below so that we have two different DataFrames containing errors: `fps` for false positibes and `fns` for false negatives.

In [ ]:
__ = df_train.query("document_classification == 0 & nlp_document_classification == 1")

In [ ]:
__ = df_train.query("document_classification == 1 & nlp_document_classification == 0")

In [ ]:
fps

In [ ]:
w_fps = MedspaCyVisualizerWidget(list(fps["doc"]))

In [ ]:
w_fns = MedspaCyVisualizerWidget(list(fns["doc"]))

## 3. Final evaluation
Once you feel like you're ready, read in the testing data, run your NLP on it, and evaluate it. You should do this **one time** so that it is an honest evaluation of how your system will perform on new, unseen data. Once you see the final results, go through the steps we did above with the training data to understand our performance on the testing set and what sorts of errors happened. How did your final system do?

In [46]:
df_test = read_pneumonia_data("test")
docs_test = list(nlp.pipe(df_test["text"]))
df_test = add_document_classifications(df_test, docs_test, classify_pna)

Reading data from: ../data/pneumonia_data_test.json


In [48]:
df_test.query("document_classification != nlp_document_classification")

,filename,text,document_classification,baseline_document_classification,nlp_document_classification,doc
1,subject_id_7234_hadm_id_17497,\n\n\n DATE: [**2977-1-16**] 11:34 PM\n ...,0,1,1,"(\n\n\n , DATE, :, [, *, *, 2977, -, 1, -,..."
5,subject_id_106_hadm_id_24012,\n\n\n DATE: [**2993-7-25**] 4:58 PM\n ...,0,0,1,"(\n\n\n , DATE, :, [, *, *, 2993, -, 7, -,..."
11,subject_id_7234_hadm_id_17497,\n\n\n DATE: [**2977-1-16**] 11:34 PM\n ...,0,1,1,"(\n\n\n , DATE, :, [, *, *, 2977, -, 1, -,..."
17,subject_id_106_hadm_id_24012,\n\n\n DATE: [**2993-7-25**] 4:58 PM\n ...,0,0,1,"(\n\n\n , DATE, :, [, *, *, 2993, -, 7, -,..."
49,subject_id_8311_hadm_id_9245,\n\n\n DATE: [**2741-5-16**] 1:46 PM\n ...,0,1,1,"(\n\n\n , DATE, :, [, *, *, 2741, -, 5, -,..."
50,subject_id_7728_hadm_id_15,\n\n\n DATE: [**3491-4-10**] 11:22 AM\n ...,0,1,1,"(\n\n\n , DATE, :, [, *, *, 3491, -, 4, -,..."
55,subject_id_7728_hadm_id_15,\n\n\n DATE: [**3491-4-10**] 11:22 AM\n ...,0,1,1,"(\n\n\n , DATE, :, [, *, *, 3491, -, 4, -,..."
56,subject_id_8311_hadm_id_9245,\n\n\n DATE: [**2741-5-16**] 1:46 PM\n ...,0,1,1,"(\n\n\n , DATE, :, [, *, *, 2741, -, 5, -,..."


In [47]:
evaluate_system(df_test)

,system,accuracy,precision,recall,f1-score,n_positive,n_total
0,baseline,0.833333,0.800000,0.857143,0.827586,28,60
1,nlp,0.866667,0.777778,1.000000,0.875000,28,60
